In [46]:
import pandas as pd

In [47]:
%run helper_functions.ipynb
%run creating_Dataframes.ipynb

**Reading the fixtures**

In [48]:
current_gw = 29
gw_fixtures = pd.read_csv(path/'fixtures.csv')
gw_fixtures = gw_fixtures[gw_fixtures['gw']>= current_gw]

**Merging with 'teams' DF to get team codes in 'gw_fixtures' and keeping relevant columns and renaming the columns**

In [49]:
# merging with teams DF to get team codes in fixtures
gw_fixtures = gw_fixtures.merge(teams, left_on = 'home_team', right_on = 'team', how ='left')
gw_fixtures = gw_fixtures.merge(teams, left_on = 'away_team', right_on = 'team', how ='left')

# keeping relevant columns and renaming the columns
gw_fixtures = gw_fixtures[['gw', 'home_team', 'away_team','team_code_x', 'team_code_y']]
gw_fixtures = gw_fixtures.rename(index = str,
                                 columns = {'team_code_x':'home_team_code',
                                            'team_code_y':'away_team_code'})

In [50]:
# gw_fixtures

**creating DF for home teams**

In [51]:
#creating DF for home teams
home_df = gw_fixtures.merge(all_players, left_on = 'home_team_code', right_on = 'team_1920',how = 'left')
#keeping relevant columns
home_df = home_df[['gw','home_team', 'away_team','full_name','position_1920', 'cost_1920', 'play_proba_1920']]
#reanming columns
home_df = home_df.rename(index = str,
                        columns = {'home_team': 'team',
                                  'away_team': 'opponent_team',
                                  'full_name':'player',
                                  'position_1920':'position',
                                  'cost_1920':'price',
                                  'play_proba_1920':'play_proba'})

#setting the flag for 'was_home' to True
home_df['was_home'] =  True

**creating DF for away teams**

In [52]:
#creating DF for home teams
away_df = gw_fixtures.merge(all_players, left_on = 'away_team_code', right_on = 'team_1920',how = 'left',sort = False)
#keeping relevant columns
away_df = away_df[['gw','home_team', 'away_team','full_name','position_1920', 'cost_1920', 'play_proba_1920']]
#reanming columns
away_df = away_df.rename(index = str,
                        columns = {'away_team': 'team',
                                   'home_team': 'opponent_team',
                                   'full_name':'player',
                                   'position_1920':'position',
                                   'cost_1920':'price',
                                   'play_proba_1920':'play_proba'})

#setting the flag for 'was_home' to True
away_df['was_home'] =  False

**Getting DF for remaining season**

In [53]:
remaining_season_df = home_df.append(away_df).reset_index(drop = True) #check this, might cause a bug
remaining_season_df['season'] = '1920'

#dividing 'price' by 10 to get real price
remaining_season_df['price'] = remaining_season_df['price']/10

#set availibilty chance to 
# 0 for 0%, 25 for 25%, 50 for 50%, 75 for 75%, 100 for 100%
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype(float)/100

#multiplying 'play_proba' with 90 to get expected minutes
remaining_season_df['minutes'] = 90 * remaining_season_df['play_proba']

#setting 'position' column to int type
remaining_season_df['position'] = remaining_season_df['position'].astype(int)



**Combining the xG dataset with the reamining season data**

In [54]:
xg_dataset = pd.read_csv(r'C:\Users\Asus\PycharmProjects\FPL Project\my_data\all_season_xg_data.csv', index_col=0)

**Converting 'season' column to string from int**

In [55]:
xg_dataset['season'] = xg_dataset['season'].astype(str)

**Concetanating the xG dataset with the reamining season dataset**

In [56]:
remaining_season_df = remaining_season_df.merge(xg_dataset, on = ['team','season'], how = 'left')

**Saving as csv**

In [57]:
remaining_season_df.to_csv(r'C:\Users\Asus\PycharmProjects\FPL Project\my_data\remaining_season.csv')
# remaining_season_df.to_excel(r'C:\Users\Asus\PycharmProjects\FPL Project\my_data\remaining_season.xlsx')

In [58]:
remaining_season_df[['gw','season']].head

<bound method NDFrame.head of       gw season
0     30   1920
1     30   1920
2     30   1920
3     30   1920
4     30   1920
...   ..    ...
6673  38   1920
6674  38   1920
6675  38   1920
6676  38   1920
6677  38   1920

[6678 rows x 2 columns]>

In [59]:
remaining_season_df.columns

Index(['gw', 'team', 'opponent_team', 'player', 'position', 'price',
       'play_proba', 'was_home', 'season', 'minutes', 'xG', 'xGA', 'npxG',
       'npxGA'],
      dtype='object')